# Language Modeling Lab: From N-grams to Neural LMs

This notebook mirrors the lecture progression for the **Language Models** part of the course.

It is structured around:

1. **Sparsity → preprocessing → n-grams → smoothing**
2. **Neural window models → embeddings → better smoothing**
3. **Reflections** that connect these models to ideas about RNNs and attention (without implementing them).

We will work with:

- The **Reuters** newswire corpus (from NLTK)
- **MLE unigram and bigram** language models
- **UNK smoothing** and **interpolated n-grams**
- A **TF–IDF pseudo-LM** (bag-of-words view as a language model)
- A **neural language model** with **pretrained word embeddings**
- **Perplexity** as a common evaluation metric
- Simple **text generation** from the models
- Short **reflection prompts** tying back to the lecture


## 1. Setup and Reuters Corpus

We start by:

- Importing libraries
- Downloading and loading the Reuters corpus from NLTK
- Creating a train / validation split
- Building a tokenization and preprocessing pipeline
- Introducing `<UNK>` to handle rare words and mitigate sparsity

In [1]:
# Import necessary libraries
import math  # For logarithm calculations in perplexity
import random  # For shuffling data and generating text
from collections import Counter, defaultdict  # For counting words efficiently

import nltk  # Natural Language Toolkit
from nltk.corpus import reuters  # Reuters news corpus
from nltk import word_tokenize  # Splits text into words

# Download required NLTK data (only needed the first time)
nltk.download("reuters")
nltk.download("punkt")

# Set random seed for reproducibility (same results every time)
random.seed(42)


def get_reuters_sentences(min_len=3, max_len=40):
    """Return tokenized sentences from the Reuters corpus.
    
    We filter sentences by length to focus on typical newswire sentences
    and avoid very short fragments or extremely long paragraphs.
    """
    sentences = []
    for file_id in reuters.fileids():
        raw = reuters.raw(file_id)
        for line in raw.split('\n'):
            line = line.strip()
            if not line:
                continue
            tokens = word_tokenize(line)
            # Keep only sentences within the desired length range
            if min_len <= len(tokens) <= max_len:
                sentences.append(tokens)
    return sentences


# Load all sentences from the Reuters corpus
sentences = get_reuters_sentences()
print(f"Collected {len(sentences)} sentences from Reuters.")


# Split data into training and validation sets (90% train, 10% validation)
random.shuffle(sentences)  # Randomize order to avoid any systematic bias
split_idx = int(0.9 * len(sentences))  # Calculate the 90% cutoff point
train_sents = sentences[:split_idx]  # First 90% for training
valid_sents = sentences[split_idx:]  # Last 10% for validation
print(f"Train sentences: {len(train_sents)}, validation sentences: {len(valid_sents)}")


def build_vocab(train_sents, min_freq=3):
    """Build vocabulary from training sentences, applying an UNK threshold.
    
    Words appearing fewer than min_freq times are excluded from the vocabulary
    and will be replaced with <UNK> during preprocessing.
    """
    # Counter is a special dictionary that counts how many times each word appears
    freq = Counter()
    
    # Loop through all training sentences
    for sent in train_sents:
        # Count each word (after converting to lowercase)
        freq.update(token.lower() for token in sent)
    
    # Start with special tokens (used for sentence boundaries and rare words)
    vocab = {"<UNK>", "<BOS>", "<EOS>"}
    
    # Add words that appear frequently enough
    for w, c in freq.items():  # w = word, c = count
        if c >= min_freq:  # Only keep words that appear at least min_freq times
            vocab.add(w)
    
    # Return both the vocabulary set and the frequency counts
    return vocab, freq


# Build vocabulary from training data
vocab, freq = build_vocab(train_sents)
print(f"Vocab size (including special tokens): {len(vocab)}")


def preprocess_sentence(tokens, vocab):
    """Lowercase, map rare tokens to <UNK>, add sentence boundary tokens."""
    # Start with beginning-of-sentence marker
    processed = ["<BOS>"]
    
    # Process each token in the sentence
    for tok in tokens:
        tok = tok.lower()  # Convert to lowercase
        
        # If word is not in our vocabulary, replace with <UNK>
        if tok not in vocab:
            tok = "<UNK>"
        
        processed.append(tok)
    
    # End with end-of-sentence marker
    processed.append("<EOS>")
    return processed


# Apply preprocessing to all training and validation sentences
# List comprehension: [function(item) for item in list] creates a new list
train_proc = [preprocess_sentence(s, vocab) for s in train_sents]
valid_proc = [preprocess_sentence(s, vocab) for s in valid_sents]

# Show an example of what a preprocessed sentence looks like
print("Example preprocessed sentence:")
print(train_proc[0])

[nltk_data] Downloading package reuters to /Users/julio/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /Users/julio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Collected 163625 sentences from Reuters.
Train sentences: 147262, validation sentences: 16363
Vocab size (including special tokens): 17645
Example preprocessed sentence:
['<BOS>', 'the', 'peak', 'broke', 'a', 'steady', 'trading', 'trend', 'of', '15', 'to', '17', 'billion', '<EOS>']


### Understanding the Preprocessing Pipeline

The code above implements several key preprocessing steps to handle **data sparsity**:

1. **Vocabulary building**: We only keep words that appear at least `min_freq` times in training data
2. **Special tokens**:
   - `<UNK>`: Replaces rare words (those not in our vocabulary)
   - `<BOS>` and `<EOS>`: Mark sentence boundaries, helping the model learn beginnings and endings
3. **Lowercasing**: Reduces sparsity by treating "The" and "the" as the same word

**Why is this important?** Without these steps, our language model would see thousands of words that appear only once or twice, making it impossible to learn meaningful patterns from limited data.


### What is Maximum Likelihood Estimation (MLE)?

MLE language models estimate probabilities by simply **counting occurrences** in the training data:

- **Unigram**: $P(w) = \frac{\text{count}(w)}{\text{total words}}$
- **Bigram**: $P(w_t \mid w_{t-1}) = \frac{\text{count}(w_{t-1}, w_t)}{\text{count}(w_{t-1})}$

This is the "maximum likelihood" approach because it assigns the highest probability to the patterns we observed most frequently. It's simple and intuitive, but has a major flaw: if we never saw a word pair in training, the probability is zero!


## 2. MLE Unigram and Bigram Language Models

We now build **maximum-likelihood** unigram and bigram LMs:

- Count how often each word and word pair appears in the training data
- Convert counts into conditional probabilities
- Use these probabilities to compute sentence likelihoods
- Evaluate via **perplexity**
- Sample simple continuations from the model

In [2]:
from typing import Any


class UnigramLM:
    """Unigram Language Model: predicts words based only on their overall frequency."""
    
    def __init__(self):
        # Initialize data structures to store word statistics
        self.counts = Counter[Any]()  # Dictionary storing how many times each word appears
        self.total = 0  # Total number of words seen
        self.vocab = set[Any]()  # Set of all unique words

    def fit(self, sentences):
        """Train the model by counting words in the training sentences."""
        # Loop through each sentence
        for sent in sentences:
            # Loop through each word in the sentence
            for w in sent:
                self.counts[w] += 1  # Increment count for this word
                self.total += 1  # Increment total word count
                self.vocab.add(w)  # Add to vocabulary set

    def prob(self, w):
        """Calculate probability of a word: P(w) = count(w) / total_words"""
        # Divide word count by total to get probability
        return self.counts[w] / self.total if self.total > 0 else 0.0

    def sent_log_prob(self, sent):
        """Calculate log probability of a sentence: sum of log probabilities of each word.
        
        We use logs because multiplying many small probabilities can cause numerical issues.
        Instead of P(w1) * P(w2) * P(w3), we compute log(P(w1)) + log(P(w2)) + log(P(w3))
        """
        log_p = 0.0  # Initialize sum to zero
        
        # Calculate log probability for each word
        for w in sent:
            p = self.prob(w)  # Get probability of this word
            if p == 0.0:
                return float("-inf")  # log(0) is undefined, return negative infinity
            log_p += math.log(p)  # Add log of probability
        
        return log_p

    def generate(self, max_len=20):
        """Generate a random sentence by sampling words according to their probabilities."""
        words = ["<BOS>"]  # Start with beginning-of-sentence token
        
        # Generate up to max_len words
        for _ in range(max_len):
            # Sample a word proportional to its frequency
            word_list = list(self.counts.keys())  # Get all words we've seen
            weights = [self.counts[w] for w in word_list]  # Get their counts as weights
            next_word = random.choices(word_list, weights=weights, k=1)[0]  # Pick one word
            words.append(next_word)
            
            # Stop if we generate the end-of-sentence token
            if next_word == "<EOS>":
                break
        
        return words


class BigramLM:
    """Bigram Language Model: predicts next word based on the previous word.
    
    P(w_next | w_prev) = count(w_prev, w_next) / count(w_prev)
    """
    
    def __init__(self, vocab):
        # defaultdict(Counter) creates a dictionary where each value is automatically a Counter
        # This lets us store bigram_counts[word1][word2] = count
        self.bigram_counts = defaultdict(Counter)  # Counts for word pairs
        self.unigram_counts = Counter()  # Counts for individual words
        self.vocab = vocab  # Vocabulary set

    def fit(self, sentences):
        """Train the model by counting word pairs (bigrams) in training sentences."""
        # Loop through each sentence
        for sent in sentences:
            # Loop through consecutive word pairs (bigrams)
            for i in range(len(sent) - 1):
                w = sent[i]  # Current word
                w_next = sent[i + 1]  # Next word
                self.unigram_counts[w] += 1  # Count current word
                self.bigram_counts[w][w_next] += 1  # Count the word pair
            
            # Don't forget to count the last word (it has no successor)
            self.unigram_counts[sent[-1]] += 1

    def prob(self, w_next, w):
        """Calculate P(w_next | w) = count(w, w_next) / count(w)"""
        denom = self.unigram_counts[w]  # How many times did we see w?
        if denom == 0:
            return 0.0  # Never saw this word, so probability is 0
        # Divide bigram count by unigram count to get conditional probability
        return self.bigram_counts[w][w_next] / denom

    def sent_log_prob(self, sent):
        """Calculate log probability of a sentence using bigram probabilities."""
        log_p = 0.0  # Initialize sum
        
        # Loop through consecutive word pairs
        for i in range(len(sent) - 1):
            w = sent[i]  # Current word
            w_next = sent[i + 1]  # Next word
            p = self.prob(w_next, w)  # Get P(next | current)
            if p == 0.0:
                return float("-inf")  # Unseen bigram
            log_p += math.log(p)  # Add log probability
        
        return log_p

    def generate(self, max_len=20):
        """Generate text by predicting each word based on the previous word."""
        cur = "<BOS>"  # Start with beginning-of-sentence token
        words = [cur]  # List to store generated words
        
        # Generate up to max_len words
        for _ in range(max_len):
            # Get all words that can follow the current word
            next_candidates = self.bigram_counts[cur]
            if not next_candidates:
                break  # No continuations found, stop generating
            
            # Sample next word based on bigram probabilities
            word_list = list(next_candidates.keys())  # Possible next words
            weights = [next_candidates[w] for w in word_list]  # Their counts
            cur = random.choices(word_list, weights=weights, k=1)[0]  # Pick one
            words.append(cur)
            
            # Stop if we generate end-of-sentence
            if cur == "<EOS>":
                break
        
        return words


def perplexity(model, sentences, use_bigrams=False):
    """Calculate perplexity: a measure of how well the model predicts the data.
    
    Lower perplexity = better model. Perplexity is roughly "how many words the model 
    is confused between at each step". For example, perplexity of 100 means the model
    is as uncertain as if it had to choose uniformly among 100 words.
    """
    total_log_prob = 0.0  # Sum of log probabilities across all sentences
    total_tokens = 0  # Total number of words/tokens
    
    # Calculate log probability for each sentence
    for sent in sentences:
        log_p = model.sent_log_prob(sent)  # Get sentence log probability
        
        # Count tokens differently for unigrams vs bigrams
        if use_bigrams:
            # For bigrams, we predict len(sent)-1 words (each word after the first)
            n_tokens = max(len(sent) - 1, 1)
        else:
            # For unigrams, we predict all words
            n_tokens = len(sent)
        
        total_log_prob += log_p
        total_tokens += n_tokens
    
    # Calculate average log probability per token
    avg_log_p = total_log_prob / total_tokens
    
    # Convert to perplexity: exp(-average_log_prob)
    return math.exp(-avg_log_p)


# Create and train a unigram model
uni_lm = UnigramLM()
uni_lm.fit(train_proc)

# Create and train a bigram model
bi_lm = BigramLM(vocab=vocab)
bi_lm.fit(train_proc)

# Evaluate both models on training data
print("Unigram perplexity (train):", perplexity(uni_lm, train_proc))
print("Bigram perplexity (train):", perplexity(bi_lm, train_proc, use_bigrams=True))

# Generate examples from unigram model
# Note: Unigram output looks very random because it only considers word frequency,
# ignoring all context and word order
print("\nGenerated sentences (unigram):")
print("  Example 1:", " ".join(uni_lm.generate()))
print("  Example 2:", " ".join(uni_lm.generate()))

# Generate examples from bigram model
# Note: Bigram output shows local coherence (reasonable word pairs) but may lack
# long-range structure and can get stuck in loops or dead-ends
print("\nGenerated sentences (bigram):")
print("  Example 1:", " ".join(bi_lm.generate()))
print("  Example 2:", " ".join(bi_lm.generate()))

Unigram perplexity (train): 461.532203786521
Bigram perplexity (train): 58.73253378585444

Generated sentences (unigram):
  Example 1: <BOS> a the <UNK> <BOS> . the resources billion inc writedowns and a the essentially <UNK> by <UNK> <EOS>
  Example 2: <BOS> . public special with for reported six could & <EOS>

Generated sentences (bigram):
  Example 1: <BOS> much producers . <EOS>
  Example 2: <BOS> fernandez said . <EOS>


## 3. Smoothing and Interpolated N-grams

Even with `<UNK>`, many bigrams will be unseen in training. To reduce zero probabilities we:

1. Keep the MLE unigram and bigram counts.
2. Define an **interpolated model**:

\begin{align}
P(w_{t} \mid w_{t-1}) = \lambda_{1} P_{\text{unigram}}(w_{t}) + \lambda_{2} P_{\text{bigram}}(w_{t} \mid w_{t-1})
\end{align}

3. Optionally add a small add-k (Laplace) smoothing term on bigrams to avoid zero probabilities entirely.

We will:

- Implement an interpolated LM
- Explore how interpolation constants affect perplexity

### Understanding Interpolation and Add-k Smoothing

**Interpolation** combines models of different orders to balance specificity with generalization:

$$P(w_{t} \mid w_{t-1}) = \lambda_{1} P_{\text{unigram}}(w_{t}) + \lambda_{2} P_{\text{bigram}}(w_{t} \mid w_{t-1})$$

where $\lambda_1 + \lambda_2 = 1$. If we never saw the bigram $(w_{t-1}, w_t)$, we can still fall back on the unigram probability.

**Add-k smoothing** (a form of Laplace smoothing) adds a small constant $k$ to all counts:

$$P_{\text{smooth}}(w_t \mid w_{t-1}) = \frac{\text{count}(w_{t-1}, w_t) + k}{\text{count}(w_{t-1}) + k \cdot |V|}$$

This ensures no probability is exactly zero, even for unseen bigrams.


In [3]:
class InterpolatedBigramLM:
    """Interpolated Bigram Language Model: combines unigram and bigram probabilities.
    
    This helps handle unseen word pairs by falling back to unigram probabilities.
    P_interpolated = λ₁ * P_unigram + λ₂ * P_bigram
    """
    
    def __init__(self, unigram_lm, bigram_lm, lambda1=0.3, lambda2=0.7, add_k=1.0):
        # Check that lambda weights sum to 1 (they define a probability distribution)
        assert abs(lambda1 + lambda2 - 1.0) < 1e-6, "lambda1 + lambda2 must equal 1.0"
        
        # Store the two models we're interpolating
        self.uni = unigram_lm  # Unigram model
        self.bi = bigram_lm  # Bigram model
        
        # Store interpolation weights
        self.lambda1 = lambda1  # Weight for unigram (typically smaller)
        self.lambda2 = lambda2  # Weight for bigram (typically larger)
        
        self.add_k = add_k  # Smoothing constant
        self.vocab = list(unigram_lm.vocab)  # Vocabulary as a list

    def prob(self, w_next, w_prev):
        """Calculate interpolated probability: mix unigram and bigram probabilities."""
        
        # Get unigram probability: P(w_next)
        p_uni = self.uni.prob(w_next)

        # Get bigram probability with add-k smoothing: P(w_next | w_prev)
        bigram_counts = self.bi.bigram_counts[w_prev]  # All words that followed w_prev
        count = bigram_counts[w_next]  # How many times w_next followed w_prev
        total = self.bi.unigram_counts[w_prev]  # How many times w_prev appeared
        V = len(self.uni.vocab)  # Vocabulary size
        
        # Apply add-k smoothing formula
        if total > 0:
            p_bi = (count + self.add_k) / (total + self.add_k * V)
        else:
            p_bi = 1.0 / V  # Uniform distribution if we never saw w_prev

        # Combine the two probabilities using the lambda weights
        return self.lambda1 * p_uni + self.lambda2 * p_bi

    def sent_log_prob(self, sent):
        """Calculate log probability of a sentence using interpolated probabilities."""
        log_p = 0.0  # Initialize sum
        
        # Loop through consecutive word pairs
        for i in range(len(sent) - 1):
            w = sent[i]  # Current word
            w_next = sent[i + 1]  # Next word
            p = self.prob(w_next, w)  # Get interpolated probability
            log_p += math.log(p)  # Add log of probability
        
        return log_p

    def generate(self, max_len=20):
        """Generate text using the interpolated model."""
        cur = "<BOS>"  # Start with beginning-of-sentence
        words = [cur]  # List to store generated words
        
        # Generate up to max_len words
        for _ in range(max_len):
            # Compute interpolated probability for each possible next word
            # This considers the entire vocabulary
            probs = [self.prob(w_next, cur) for w_next in self.vocab]
            
            # Sample from the distribution using these probabilities
            cur = random.choices(self.vocab, weights=probs, k=1)[0]
            words.append(cur)
            
            # Stop if we generate end-of-sentence
            if cur == "<EOS>":
                break
        
        return words


# Create an interpolated model combining the unigram and bigram models
# lambda1=0.3 means 30% weight on unigram, lambda2=0.7 means 70% on bigram
interp_lm = InterpolatedBigramLM(uni_lm, bi_lm, lambda1=0.3, lambda2=0.7, add_k=0.1)

# Evaluate on validation set (data the model hasn't seen during training)
print("Interpolated bigram perplexity (validation):", perplexity(interp_lm, valid_proc, use_bigrams=True))

# Generate example sentences
# Note: Interpolated model produces smoother output than pure bigram by falling back
# to unigram probabilities, reducing the impact of unseen bigrams
print("\nGenerated sentences (interpolated):")
print("  Example 1:", " ".join(interp_lm.generate()))
print("  Example 2:", " ".join(interp_lm.generate()))

Interpolated bigram perplexity (validation): 231.8023664853118

Generated sentences (interpolated):
  Example 1: <BOS> sheldahl 71.7 veto approve purchases , <EOS>
  Example 2: <BOS> in 36 stevens modified preceding advertisement two sub-saharan depleted tractor <BOS> proceeds and is towards <BOS> would pacific telesis one-week


### Reflection 1: Sparsity, Preprocessing, and Smoothing

Discuss (in your own words, in the notebook):

1. How did preprocessing (lowercasing, `<UNK>`, `<BOS>`, `<EOS>`) change the effective sparsity of the data?
2. Compare perplexity for the unigram, bigram, and interpolated models. What do you observe?
3. How does this relate to the lecture narrative about:
   - One-hot sparsity
   - Co-occurrence counts
   - The need for better smoothing and generalization?


## 4. TF-IDF as a Pseudo Language Model

A classical bag-of-words representation like **TF-IDF** ignores word order but can still be
viewed (loosely) as a **pseudo language model** if we normalize the TF-IDF vector to obtain a
distribution over the vocabulary.

In this section we:

- Build a TF-IDF representation for Reuters documents
- Convert TF-IDF vectors into normalized distributions over terms
- Use them to score sentences and compute a pseudo perplexity
- Compare this to real n-gram LMs

In [4]:
# Import sklearn's TF-IDF implementation and numpy for numerical operations
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


def join_tokens(sent):
    """Convert a tokenized sentence back to a string, removing special tokens."""
    # TF-IDF works with text strings, so we need to join the token list
    # Skip <BOS> and <EOS> since they're not real content words
    return " ".join(tok for tok in sent if tok not in {"<BOS>", "<EOS>"})


# Convert all preprocessed sentences back to strings
train_docs = [join_tokens(s) for s in train_proc]  # Training documents as strings
valid_docs = [join_tokens(s) for s in valid_proc]  # Validation documents as strings

# Create TF-IDF vectorizer
# TF-IDF = Term Frequency - Inverse Document Frequency
# It weights words by: (how often in this doc) / (how common across all docs)
# min_df=5 means ignore words that appear in fewer than 5 documents
vectorizer = TfidfVectorizer(min_df=5)

# Fit the vectorizer on training data and compute TF-IDF scores
# This learns the vocabulary and IDF values
X_train = vectorizer.fit_transform(train_docs)

# Extract vocabulary and create index mapping
vocab_list = vectorizer.get_feature_names_out()  # List of all words
vocab_index = {w: i for i, w in enumerate(vocab_list)}  # Map word -> position in vector

print("TF-IDF vocabulary size:", len(vocab_list))


def tfidf_to_prob(vec):
    """Convert a sparse TF-IDF vector into a normalized probability distribution.
    
    TF-IDF weights aren't probabilities, but we can normalize them to sum to 1.
    This lets us use them as a pseudo-probability distribution over words.
    """
    # Convert sparse matrix to a regular numpy array and flatten to 1D
    vec = np.asarray(vec.todense()).flatten()
    
    # Calculate sum of all TF-IDF weights
    total = vec.sum()
    
    if total == 0.0:
        # If all weights are zero, return uniform distribution (all words equally likely)
        return np.ones_like(vec) / len(vec)
    
    # Normalize: divide each weight by total so they sum to 1
    return vec / total


def sentence_log_prob_tfidf(sent, prob_vec):
    """Score a sentence by summing log probabilities of its words.
    
    This treats the TF-IDF distribution as a language model and scores how
    likely the sentence is under that distribution.
    """
    log_p = 0.0  # Initialize sum
    
    # Loop through each word in the sentence
    for w in sent:
        # Skip special tokens (not real content words)
        if w in {"<BOS>", "<EOS>"}:
            continue
        
        # Find the position of this word in the TF-IDF vocabulary
        idx = vocab_index.get(w, None)
        
        if idx is None:
            # Word not in TF-IDF vocabulary, skip it
            continue
        
        # Get probability for this word
        p = prob_vec[idx]
        
        if p > 0.0:
            log_p += math.log(p)  # Add log probability
    
    return log_p


def perplexity_tfidf(docs, sents):
    """Calculate perplexity using TF-IDF as a pseudo language model.
    
    For each sentence, we:
    1. Compute its TF-IDF vector
    2. Convert to a probability distribution
    3. Score the sentence under that distribution
    """
    total_log_prob = 0.0  # Sum of log probabilities
    total_tokens = 0  # Total number of words scored
    
    # Process each document-sentence pair together
    # zip pairs up corresponding elements from two lists
    for doc_text, sent in zip(docs, sents):
        # Compute TF-IDF vector for this document
        vec = vectorizer.transform([doc_text])
        
        # Convert TF-IDF weights to probabilities
        prob_vec = tfidf_to_prob(vec)
        
        # Score the sentence using these probabilities
        log_p = sentence_log_prob_tfidf(sent, prob_vec)
        
        # Count non-special tokens (actual content words)
        n_tokens = sum(1 for w in sent if w not in {"<BOS>", "<EOS>"})
        
        # Accumulate totals
        total_log_prob += log_p
        total_tokens += max(n_tokens, 1)  # Avoid division by zero
    
    # Calculate average log probability per token
    avg_log_p = total_log_prob / total_tokens
    
    # Convert to perplexity
    return math.exp(-avg_log_p)


# Evaluate TF-IDF pseudo-LM on validation data
print("TF-IDF pseudo perplexity (validation):", perplexity_tfidf(valid_docs, valid_proc))

TF-IDF vocabulary size: 10238
TF-IDF pseudo perplexity (validation): 5.766263093773351


### Why TF-IDF as a "Language Model"?

TF-IDF is fundamentally a **bag-of-words** representation that ignores word order. However, we can interpret it as a pseudo language model by:

1. Computing TF-IDF weights for each word in a document
2. Normalizing these weights to sum to 1, creating a probability distribution
3. Using this distribution to score sentences (higher probability for words with high TF-IDF)

**Key difference from n-grams:** TF-IDF doesn't model word order or context, so it captures **what** words appear but not **how** they're arranged. This makes it a poor true language model, but useful for comparison to see the importance of modeling sequential structure.


In [5]:
def generate_tfidf_pseudo(max_len=20, seed_doc_idx=0):
    """Generate 'text' from TF-IDF (a hacky demonstration).
    
    WARNING: This is NOT a real language model! TF-IDF has no sequential structure.
    This function artificially samples words from a TF-IDF distribution just to show
    what happens when we ignore word order entirely.
    
    The output will look very random and incoherent because TF-IDF only knows
    which words are important, not how words should be ordered or which words
    typically follow each other.
    """
    # Get TF-IDF vector for a seed document
    seed_doc = train_docs[seed_doc_idx]
    vec = vectorizer.transform([seed_doc])
    prob_vec = tfidf_to_prob(vec)
    
    # Sample words randomly according to TF-IDF weights (ignoring all order!)
    words = ["<BOS>"]
    for _ in range(max_len):
        # Sample a word based on its TF-IDF weight
        idx = np.random.choice(len(prob_vec), p=prob_vec)
        word = vocab_list[idx]
        words.append(word)
        
        # Arbitrary stopping condition (just for demonstration)
        if len(words) > 5 and random.random() < 0.2:
            break
    
    words.append("<EOS>")
    return words


# Generate examples to show the lack of sequential coherence
print("TF-IDF 'generated' text (NOT a real language model!):")
print("Note: This looks incoherent because TF-IDF is a bag-of-words model")
print("that doesn't capture word order or sequential dependencies.\n")

# Generate from different seed documents to show variety
print("  Example 1:", " ".join(generate_tfidf_pseudo(max_len=15, seed_doc_idx=0)))
print("  Example 2:", " ".join(generate_tfidf_pseudo(max_len=15, seed_doc_idx=100)))

print("\nCompare these 'sentences' to the n-gram outputs above.")
print("The n-grams produce locally coherent text because they model word sequences.")
print("TF-IDF cannot do this - it only knows which words are topically important.")


TF-IDF 'generated' text (NOT a real language model!):
Note: This looks incoherent because TF-IDF is a bag-of-words model
that doesn't capture word order or sequential dependencies.

  Example 1: <BOS> trading of steady peak billion <EOS>
  Example 2: <BOS> 300 mark to 300 300 <EOS>

Compare these 'sentences' to the n-gram outputs above.
The n-grams produce locally coherent text because they model word sequences.
TF-IDF cannot do this - it only knows which words are topically important.


### Reflection 2: Bag-of-Words vs Sequence Models

1. In what sense does a TF-IDF vector act like a **language model**? In what sense does it clearly not?
2. Compare TF-IDF pseudo perplexity to n-gram perplexities. Why is this comparison imperfect?
3. How do these observations match the lecture discussion about:
   - Bag-of-words and TF-IDF
   - Co-occurrence matrices
   - The move toward distributed representations?

## 5. Neural Window Language Model with Pretrained Embeddings

We now move to a **neural language model** that:

- Uses a **fixed window** of context words
- Represents each word with a **pretrained embedding**
- Predicts the next word using a small feedforward neural network

This corresponds to the lecture's transition from sparse n-grams to **dense, distributed representations**.

Implementation notes:

- For simplicity, we restrict to a small subset of the vocabulary.
- We assume access to pretrained embeddings (for example, via `gensim.downloader` or local GloVe files).
- Training on the full Reuters corpus can be slow; you may want to use a small sample when running this yourself.

### Understanding Window-Based Neural Language Models

Unlike n-grams that use sparse one-hot vectors, neural LMs use **dense word embeddings**:

- Each word is represented as a continuous vector (e.g., 100 dimensions)
- Similar words have similar vectors, enabling **generalization**
- A fixed window of context words is concatenated and fed through a neural network

**Architecture overview:**
1. Look up embeddings for the last $n$ words (the context window)
2. Concatenate these embeddings into a single vector
3. Pass through feedforward layers to predict the next word

**Key advantage:** If the model has seen "the cat sat" and "the dog sat", it can generalize to "the kitten sat" because cat, dog, and kitten have similar embeddings.


In [7]:
# Import PyTorch - a deep learning library
import torch  # Main PyTorch module
import torch.nn as nn  # Neural network components
import torch.optim as optim  # Optimization algorithms (like gradient descent)

# Configuration parameters
CONTEXT_SIZE = 3  # How many previous words to look at when predicting the next word
MAX_VOCAB_NEURAL = 5000  # Limit vocabulary size for faster training

# Build a smaller vocabulary for the neural model (use most frequent words)
# Reserve space for special tokens by getting MAX_VOCAB_NEURAL - 3 most common words
most_common = freq.most_common(MAX_VOCAB_NEURAL - 3)  # Get top 4997 most frequent words
neural_vocab = [w for w, _ in most_common]  # Extract just the words (ignore counts)

# Add special tokens to the vocabulary (now we have room for them)
for special in ["<UNK>", "<BOS>", "<EOS>"]:
    if special not in neural_vocab:
        neural_vocab.append(special)  # Add if missing

# Create word-to-index and index-to-word dictionaries
# Neural networks work with numbers, not words, so we need this mapping
neural_stoi = {w: i for i, w in enumerate(neural_vocab)}  # string to index
neural_itos = {i: w for w, i in neural_stoi.items()}  # index to string

vocab_size_neural = len(neural_vocab)
print("Neural LM vocab size:", vocab_size_neural)


def sent_to_indices(sent):
    """Convert a sentence (list of words) to a list of integer indices."""
    indices = []
    for w in sent:
        # If word not in our neural vocabulary, use <UNK>
        if w not in neural_stoi:
            w = "<UNK>"
        indices.append(neural_stoi[w])  # Look up the index for this word
    return indices


def build_window_dataset(sentences, context_size=CONTEXT_SIZE, max_sents=5000):
    """Create training examples: each example is (context_words, target_word).
    
    For a sentence "the cat sat on mat", with context_size=2:
    - Example 1: context=[the, cat], target=sat
    - Example 2: context=[cat, sat], target=on
    - Example 3: context=[sat, on], target=mat
    """
    # Lists to store all training examples
    contexts = []  # Each context is a list of context_size word indices
    targets = []   # Each target is a single word index to predict
    
    # Process up to max_sents sentences (to keep dataset manageable)
    for sent in sentences[:max_sents]:
        # Convert words to indices
        word_indices = sent_to_indices(sent)
        
        # Slide a window across the sentence to create training examples
        # Start at position context_size (we need context_size previous words)
        for i in range(context_size, len(word_indices)):
            # Get the context: previous context_size words
            context_window = word_indices[i - context_size : i]
            
            # Get the target: the word we're trying to predict
            next_word = word_indices[i]
            
            # Add this training example
            contexts.append(context_window)
            targets.append(next_word)
    
    # Convert lists to PyTorch tensors
    # dtype=torch.long means integers (required for indexing embeddings)
    return torch.tensor(contexts, dtype=torch.long), torch.tensor(targets, dtype=torch.long)


# Build the training dataset
X_train_win, y_train_win = build_window_dataset(train_proc, context_size=CONTEXT_SIZE)
print("Window dataset size:", X_train_win.shape, y_train_win.shape)

Neural LM vocab size: 5000
Window dataset size: torch.Size([41851, 3]) torch.Size([41851])


### The Neural Architecture

Our model is a simple feedforward network:

```
Input: [word_{t-3}, word_{t-2}, word_{t-1}]  (context window)
   ↓
Embedding Layer: converts each word index to a dense vector
   ↓
Concatenate: [embed_{t-3} || embed_{t-2} || embed_{t-1}]
   ↓
Hidden Layer: fully connected with ReLU activation
   ↓
Output Layer: predicts probability distribution over vocabulary
   ↓
Output: P(word_t | word_{t-3}, word_{t-2}, word_{t-1})
```

The embedding layer is where the magic happens: it learns to map words to vectors such that words used in similar contexts end up close together in the embedding space.


In [8]:
class NeuralWindowLM(nn.Module):
    def __init__(self, vocab_size, embed_dim=100, context_size=3, hidden_dim=128):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.linear1 = nn.Linear(embed_dim * context_size, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        # x shape: (batch_size, context_size)
        embeds = self.embed(x)  
        # embeds shape: (batch_size, context_size, embed_dim)
        
        # Flatten the embeddings for each example
        embeds = embeds.view(embeds.size(0), -1)  
        # embeds shape: (batch_size, context_size * embed_dim)
        
        # Pass through hidden layer
        h = self.relu(self.linear1(embeds))
        # h shape: (batch_size, hidden_dim)
        
        # Generate logits for each word in vocabulary
        logits = self.linear2(h)
        # logits shape: (batch_size, vocab_size)
        
        return logits


In [9]:
# Create model instance and training components
# Choose device: use GPU if available (faster), otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create the neural language model
model = NeuralWindowLM(
    vocab_size_neural,  # Number of words in vocabulary
    embed_dim=100,  # Size of word embedding vectors
    context_size=CONTEXT_SIZE,  # Number of context words (3)
    hidden_dim=128  # Size of hidden layer
).to(device)  # Move model to GPU/CPU

# Loss function: measures how wrong the predictions are
# CrossEntropyLoss is standard for classification (predicting which word comes next)
criterion = nn.CrossEntropyLoss()

# Optimizer: updates model weights to reduce loss
# Adam is a popular optimization algorithm (improved gradient descent)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


def train_neural_lm(model, X, y, epochs=1, batch_size=256):
    """Train the neural language model on the provided dataset.
    
    Args:
        model: The neural network to train
        X: Input data (context windows)
        y: Target data (next words)
        epochs: Number of times to iterate through the entire dataset
        batch_size: Number of examples to process at once
    """
    model.train()  # Put model in training mode
    num_examples = X.size(0)  # Total number of training examples
    
    # Loop through epochs (complete passes through the data)
    for epoch in range(epochs):
        # Shuffle data each epoch to avoid learning order-dependent patterns
        shuffle_indices = torch.randperm(num_examples)  # Random permutation of indices
        X_shuffled = X[shuffle_indices]  # Shuffle inputs
        y_shuffled = y[shuffle_indices]  # Shuffle targets the same way
        
        epoch_loss = 0.0  # Track total loss for this epoch
        num_batches = 0  # Count number of batches
        
        # Process data in batches (more efficient than one example at a time)
        for batch_start in range(0, num_examples, batch_size):
            batch_end = batch_start + batch_size
            
            # Get this batch of data and move to device (GPU/CPU)
            X_batch = X_shuffled[batch_start:batch_end].to(device)
            y_batch = y_shuffled[batch_start:batch_end].to(device)
            
            # Forward pass: compute predictions
            optimizer.zero_grad()  # Clear gradients from previous step
            logits = model(X_batch)  # Get model predictions
            loss = criterion(logits, y_batch)  # Calculate loss
            
            # Backward pass: compute gradients and update weights
            loss.backward()  # Compute gradients (how to adjust weights)
            optimizer.step()  # Update weights using gradients
            
            # Track statistics
            epoch_loss += loss.item()
            num_batches += 1
        
        # Print average loss for this epoch
        avg_loss = epoch_loss / num_batches
        print(f"Epoch {epoch + 1}/{epochs}, Average Loss: {avg_loss:.4f}")


# Train the neural language model
# Note: This takes approximately 2-3 minutes on a recent MacBook (CPU)
print("Training neural language model...")
train_neural_lm(model, X_train_win, y_train_win, epochs=3)


Training neural language model...
Epoch 1/3, Average Loss: 6.0034
Epoch 2/3, Average Loss: 4.8322
Epoch 3/3, Average Loss: 4.4309


### 5.1 Perplexity and Text Generation for the Neural LM

We now:

- Define a perplexity function for the neural LM
- Implement a simple left-to-right text generation procedure using the fixed window context

Note that this is still a **window-based** model, not an RNN or transformer.

In [10]:
def neural_sent_log_prob(model, sent, context_size=CONTEXT_SIZE):
    """Calculate log probability of a sentence using the neural language model.
    
    We slide a context window across the sentence and predict each word
    given the previous context_size words.
    """
    model.eval()  # Put model in evaluation mode (turns off dropout, etc.)
    
    # Convert sentence words to integer indices
    idxs = sent_to_indices(sent)
    
    # If sentence is too short, can't make predictions
    if len(idxs) <= context_size:
        return 0.0
    
    log_p = 0.0  # Initialize sum of log probabilities
    
    # torch.no_grad() disables gradient calculation (faster, uses less memory)
    # We don't need gradients since we're not training
    with torch.no_grad():
        # Slide window across sentence, predicting each word
        for i in range(context_size, len(idxs)):
            # Get context: the previous context_size words
            context = torch.tensor([idxs[i - context_size:i]], dtype=torch.long).to(device)
            
            # Get model predictions (logits = unnormalized scores)
            logits = model(context)
            
            # Convert logits to probabilities using softmax
            probs = torch.softmax(logits, dim=-1)
            
            # Get probability of the actual next word that appeared
            p = probs[0, idxs[i]].item()  # .item() converts to Python number
            
            if p > 0.0:
                log_p += math.log(p)  # Add log probability
    
    return log_p


def neural_perplexity(model, sentences, context_size=CONTEXT_SIZE, max_sents=1000):
    """Calculate perplexity of the neural LM on a set of sentences.
    
    Args:
        model: The neural language model
        sentences: List of tokenized sentences
        context_size: Number of context words the model uses
        max_sents: Maximum number of sentences to evaluate (for speed)
    """
    total_log_prob = 0.0  # Sum of log probabilities
    total_tokens = 0  # Total number of words predicted
    
    # Evaluate on up to max_sents sentences
    for sent in sentences[:max_sents]:
        # Get log probability for this sentence
        log_p = neural_sent_log_prob(model, sent, context_size=context_size)
        
        # Count tokens: we predict len(sent) - context_size words
        # (can't predict first context_size words, no prior context)
        n_tokens = max(len(sent) - context_size, 1)
        
        # Accumulate totals
        total_log_prob += log_p
        total_tokens += n_tokens
    
    # Calculate average log probability per token
    avg_log_p = total_log_prob / total_tokens
    
    # Convert to perplexity
    return math.exp(-avg_log_p)


def generate_neural(model, max_len=20, context_size=CONTEXT_SIZE, prefix=None):
    """Generate text using the neural language model.
    
    Args:
        model: The trained neural language model
        max_len: Maximum number of words to generate
        context_size: Number of context words the model uses
        prefix: Starting words (if None, starts with <BOS> tokens)
    
    Returns:
        List of generated words
    """
    model.eval()  # Put model in evaluation mode
    
    # If no prefix provided, start with beginning-of-sentence tokens
    if prefix is None:
        prefix = ["<BOS>"] * context_size
    
    # Start with the prefix words
    tokens = prefix[:]
    
    # No gradient calculation needed for generation
    with torch.no_grad():
        # Generate up to max_len words
        for _ in range(max_len):
            # Get the last context_size words as context
            context = tokens[-context_size:]
            
            # Convert words to indices (use <UNK> if word not in vocabulary)
            idxs = [neural_stoi.get(w, neural_stoi["<UNK>"]) for w in context]
            
            # Convert to tensor and move to device
            x = torch.tensor([idxs], dtype=torch.long).to(device)
            
            # Get model predictions
            logits = model(x)
            
            # Convert to probabilities and move to CPU as numpy array
            probs = torch.softmax(logits, dim=-1)[0].cpu().numpy()
            
            # Sample next word according to the probability distribution
            next_idx = np.random.choice(len(probs), p=probs)
            
            # Convert index back to word
            next_word = neural_itos[next_idx]
            
            # Add to generated sequence
            tokens.append(next_word)
            
            # Stop if we generate end-of-sentence
            if next_word == "<EOS>":
                break
    
    return tokens


# Evaluate and generate text from the neural LM
print("Neural LM perplexity (validation):", neural_perplexity(model, valid_proc))

# Generate example sentences
# Note: Neural LM with learned embeddings can generalize better than n-grams
# because similar words have similar representations, enabling the model to
# make more informed predictions even for unseen word combinations
print("\nGenerated sentences (neural LM):")
print("  Example 1:", " ".join(generate_neural(model)))
print("  Example 2:", " ".join(generate_neural(model)))

print("\nCompare the quality of these generated sentences to the n-gram models above.")
print("The neural model benefits from learned word embeddings that capture semantic similarity.")

Neural LM perplexity (validation): 158.8833076986575

Generated sentences (neural LM):
  Example 1: <BOS> <BOS> <BOS> income to <UNK> mln <EOS>
  Example 2: <BOS> <BOS> <BOS> the <UNK> can , are 80 <UNK> <EOS>

Compare the quality of these generated sentences to the n-gram models above.
The neural model benefits from learned word embeddings that capture semantic similarity.


### Comparing Generated Text Quality Across Models

Now that you've seen generation examples from each model, let's compare their characteristics:

| Model | Text Quality | Why? |
|-------|-------------|------|
| **Unigram** | Random, incoherent | Only considers word frequency; no context or word order |
| **Bigram** | Local coherence, but brittle | Models word pairs; gets local structure right but can't maintain long-range coherence |
| **Interpolated** | Smoother than bigram | Falls back to unigram when bigrams unseen; more robust but still limited to pairs |
| **TF-IDF** | Random, topical but incoherent | Bag-of-words: knows *what* words matter but not *how* to order them |
| **Neural** | Better generalization | Learned embeddings capture semantic similarity; can generalize to unseen contexts |

**Key insight:** As we move from sparse count-based models to dense neural representations, we get better generalization and more coherent text. However, even the neural window model is limited by its fixed context size—this is where RNNs and Transformers (with unlimited context) become important.


### Reflection 3: From N-grams to Neural Models (and Beyond)

1. In what ways does the neural window LM **reuse information** across contexts more effectively than n-grams?
2. How do **embeddings** change the sparsity story compared to one-hot vectors and raw co-occurrence counts?
3. Based on the lecture, how would **RNNs** or **attention-based models** (transformers) further extend these ideas?
   - What limitations of fixed-window models might they address?
   - How do they handle **long-range dependencies** and **variable-length context**?

Write a short paragraph connecting:

- The MLE and smoothed n-gram models
- The TF-IDF pseudo-LM
- The neural window LM
- The conceptual role of RNNs and attention discussed in the slides

## 6. Summary

In this notebook you have:

- Built and evaluated **unigram and bigram language models** with MLE.
- Applied **UNK smoothing** and **interpolated n-grams** to reduce sparsity.
- Interpreted **TF-IDF** as a pseudo language model and compared it to sequence-based LMs.
- Implemented a **neural window LM** with embeddings and discussed its evaluation through **perplexity** and **text generation**.
- Reflected on how **RNNs** and **attention** fit into the progression from sparse to dense, context-aware models.

This mirrors the lecture trajectory:

1. Sparsity, preprocessing, n-grams, smoothing
2. Neural window models and embeddings as better smoothing
3. RNNs and attention as conceptual extensions, appearing here only in reflection prompts.